# Model Context Protocol (MCP) Demo

This notebook demonstrates the **Model Context Protocol (MCP)** - an open standard for connecting AI models to external tools and data sources.

## This Demo

We'll use the **Singapore Weather MCP Server** (`d2-sgweather-mcp/`) which:
- Fetches real weather data from data.gov.sg
- Runs as a Node.js stdio MCP process
- Provides the `get_singapore_weather` tool

## Required Environment Variables

```
ANTHROPIC_API_KEY    # Your Anthropic API key (required)
```

Please refer to the [README](README.md) for instructions on setting up environment variables.

In [ ]:
# If running standalone without the project setup:
# !pip install anthropic python-dotenv

# IMPORTANT: The MCP server requires Node.js
# Install the sgweather-mcp server dependencies:
# cd sgweather-mcp && npm install

In [ ]:
import os
import json
import asyncio
from anthropic import Anthropic
from dotenv import load_dotenv
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# Load environment variables
load_dotenv(override=True)

# Initialize Anthropic client
client = Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))

print("✅ Anthropic client initialized")

## MCP Server Setup

The Singapore Weather MCP Server is located in `d2-sgweather-mcp/` and runs as a Node.js process. 

Anthropic's client launches the Node.js server as a subprocess when needed. Communication happens through standard input/output.

In [ ]:
import os

server_path = os.path.abspath("d2-sgweather-mcp/server.js")
print(f"MCP Server location: {server_path}")
print(f"Server exists: {os.path.exists(server_path)}")

import subprocess
try:
    result = subprocess.run(["node", "--version"], capture_output=True, text=True)
    print(f"Node.js version: {result.stdout.strip()}")
except FileNotFoundError:
    print("Node.js not found! Please install Node.js to run the MCP server")

## Connecting to the MCP Server

We'll connect to the MCP server using the MCP Python client. The client will:
1. Spawn the Node.js server as a subprocess
2. Communicate via stdio (standard input/output)
3. Discover available tools
4. Execute tool calls as needed

## MCP Demo: Singapore Weather

In [ ]:
# Configure the MCP server
server_path = os.path.abspath("d2-sgweather-mcp/server.js")

# MCP Server parameters
server_params = StdioServerParameters(
    command="node",
    args=[server_path],
    env=None
)

async def get_mcp_tools():
    """Connect to MCP server and get available tools"""
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            # List available tools from the MCP server
            tools_list = await session.list_tools()
            return tools_list

# Get tools from MCP server
print("\n🔍 Discovering tools from MCP server...")
tools_response = await get_mcp_tools()

print(f"\n✅ Found {len(tools_response.tools)} tool(s):")
for tool in tools_response.tools:
    print(f"  - {tool.name}: {tool.description}")

## Convert MCP Tools for Claude

Now we'll convert the MCP tool definitions into the format Claude expects:

In [ ]:
# Convert MCP tools to Claude's tool format
claude_tools = []
for tool in tools_response.tools:
    claude_tool = {
        "name": tool.name,
        "description": tool.description,
        "input_schema": tool.inputSchema
    }
    claude_tools.append(claude_tool)

print(json.dumps(claude_tools, indent=2))

# Create a function to execute tools via MCP
async def execute_mcp_tool(tool_name, tool_args):
    print(tool_name)
    print(tool_args)
    """Execute a tool via the MCP server"""
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            # Call the tool
            result = await session.call_tool(tool_name, arguments=tool_args)
            return result


# Try to execute
weather = await execute_mcp_tool("get_singapore_weather", {})
print(weather)


## Comparison: Claude Without MCP vs With MCP


In [ ]:
# Demo 1: WITHOUT MCP - Claude has no access to real-time data
async def chat_without_mcp(user_question):
    """Chat with Claude WITHOUT MCP tools"""
    messages = [
        {"role": "user", "content": user_question}
    ]

    response = client.messages.create(
        model="claude-3-5-haiku-20241022",
        max_tokens=1024,
        messages=messages
    )

    print("🤖 Claude's Response (without MCP):\n")
    for block in response.content:
      if hasattr(block, 'text'):
        print(block.text)

    return response


user_question = "What's the weather like in Singapore today? Give me a brief summary."
print(f"👤 User: {user_question}\n")
await chat_without_mcp(user_question)

In [ ]:
async def chat_with_mcp(user_message):
    """Have a conversation with Claude using MCP tools"""
    print(f"👤 User: {user_message}\n")

    # Initial request to Claude
    messages = [{"role": "user", "content": user_message}]

    response = client.messages.create(
        model="claude-3-5-haiku-20241022",
        max_tokens=1024,
        tools=claude_tools, # Notice the tools are included here
        messages=messages
    )

    # Process the response
    while response.stop_reason == "tool_use":
        # Extract tool calls
        tool_use_blocks = [block for block in response.content if block.type == "tool_use"]

        if not tool_use_blocks:
            break

        print("🔧 Claude is using MCP tools:\n")

        # Add assistant's response to messages
        messages.append({"role": "assistant", "content": response.content})

        # Execute each tool via MCP
        tool_results = []
        for tool_block in tool_use_blocks:
            tool_name = tool_block.name
            tool_args = tool_block.input

            print(f"  Tool: {tool_name}")
            print(f"  Args: {json.dumps(tool_args, indent=4)}")

            # Execute via MCP server
            result = await execute_mcp_tool(tool_name, tool_args)

            # Extract content from result
            if result.content:
                content_text = ""
                for content_item in result.content:
                    if hasattr(content_item, 'text'):
                        content_text += content_item.text

                print(f"  ✅ Result from MCP server: {content_text[:150]}...")

                tool_results.append({
                    "type": "tool_result",
                    "tool_use_id": tool_block.id,
                    "content": content_text
                })

        # Add tool results and get Claude's final response
        messages.append({"role": "user", "content": tool_results})

        response = client.messages.create(
            model="claude-3-5-haiku-20241022",
            max_tokens=1024,
            tools=claude_tools,
            messages=messages
        )

        print()

    # Display Claude's final response
    print("🤖 Claude's Response (with MCP):\n")
    for block in response.content:
        if hasattr(block, 'text'):
            print(block.text)

    return response


await chat_with_mcp("What's the weather like in Singapore today? Give me a brief summary.")

In [ ]:
umbrella_question = "Should I bring an umbrella today in Singapore? Also, what are the temperature and humidity levels?"

await chat_without_mcp(umbrella_question)

In [ ]:
print(umbrella_question)
await chat_with_mcp(umbrella_question)